In [3]:
import geemap
import ee
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [7]:
def clip(img):
    return img.clip(roi)

In [11]:
roi = Map.draw_last_feature.geometry()

In [13]:
images = ee.ImageCollection("NASA/GIMMS/3GV0").select('ndvi')\
        .filterDate('2012', '2013').map(clip)

In [15]:
visParam = {
 'min': -1,
 'max': 1,
 'palette': ['000000', 'f5f5f5', '119701']
}
Map.addLayer(images, visParam, 'NDVI')

In [17]:
img = images.toBands()
ps = img.sample(region=roi, geometries=True)

In [19]:
ps.first().getInfo()

{'type': 'Feature',
 'geometry': {'geodesic': False,
  'type': 'Point',
  'coordinates': [115.875, 29.708333333333336]},
 'id': '0',
 'properties': {'201201a_ndvi': 0.08299999684095383,
  '201201b_ndvi': 0.06700000166893005,
  '201202a_ndvi': 0.07500000298023224,
  '201202b_ndvi': 0.09799999743700027,
  '201203a_ndvi': 0.12399999797344208,
  '201203b_ndvi': 0.14800000190734863,
  '201204a_ndvi': 0.13500000536441803,
  '201204b_ndvi': 0.35499998927116394,
  '201205a_ndvi': 0.5509999990463257,
  '201205b_ndvi': 0.5849999785423279,
  '201206a_ndvi': 0.3659999966621399,
  '201206b_ndvi': 0.4560000002384186,
  '201207a_ndvi': 0.6460000276565552,
  '201207b_ndvi': 0.6809999942779541,
  '201208a_ndvi': 0.546999990940094,
  '201208b_ndvi': 0.4429999887943268,
  '201209a_ndvi': 0.5239999890327454,
  '201209b_ndvi': 0.382999986410141,
  '201210a_ndvi': 0.2800000011920929,
  '201210b_ndvi': 0.17100000381469727,
  '201211a_ndvi': 0.21400000154972076,
  '201211b_ndvi': 0.17900000512599945,
  '20121

In [21]:
Map.addLayer(ps, {}, 'ps')

In [23]:
order = 2
window_size = 11
def savitzky_golay(y):
    half_window = (window_size - 1) / 2
    deriv = 0
    order_range = ee.List.sequence(0, order)
    k_range = ee.List.sequence(-half_window, half_window)
    def fun1(k):
        return order_range.map(lambda o: ee.Number(k).pow(o))
    b = ee.Array(k_range.map(fun1))
    mPI = ee.Array(b.matrixPseudoInverse())
    impulse_response = (mPI.slice(**{'axis': 0, 'start': deriv, 'end': deriv + 1})).project([1])
    y0 = y.get(0)
    firstvals = y.slice(1, half_window + 1).reverse().map(
        lambda e: ee.Number(e).subtract(y0).abs().multiply(-1).add(y0))
    yend = y.get(-1)
    lastvals = y.slice(-half_window - 1, -1).reverse().map(
        lambda e: ee.Number(e).subtract(yend).abs().add(yend))
    y_ext = firstvals.cat(y).cat(lastvals)
    runLength = ee.List.sequence(0, y_ext.length().subtract(window_size))
    smooth = runLength.map(
        lambda i: ee.Array(y_ext.slice(ee.Number(i), ee.Number(i).add(window_size))).multiply(impulse_response).reduce("sum", [0]).get([0])
    )
    return smooth

In [25]:
def fun(feature):
    List = bands.map(lambda i: feature.get(i))
    sg_ = savitzky_golay(List)
    return feature.set(ee.Dictionary.fromLists(bands_sg, sg_))

In [27]:
bands = img.bandNames()
List = bands.map(lambda i: ps.first().get(i))

In [29]:
bands_sg = bands.map(lambda i: ee.String(i).cat('_sg'))
bands_sg.getInfo()

['201201a_ndvi_sg',
 '201201b_ndvi_sg',
 '201202a_ndvi_sg',
 '201202b_ndvi_sg',
 '201203a_ndvi_sg',
 '201203b_ndvi_sg',
 '201204a_ndvi_sg',
 '201204b_ndvi_sg',
 '201205a_ndvi_sg',
 '201205b_ndvi_sg',
 '201206a_ndvi_sg',
 '201206b_ndvi_sg',
 '201207a_ndvi_sg',
 '201207b_ndvi_sg',
 '201208a_ndvi_sg',
 '201208b_ndvi_sg',
 '201209a_ndvi_sg',
 '201209b_ndvi_sg',
 '201210a_ndvi_sg',
 '201210b_ndvi_sg',
 '201211a_ndvi_sg',
 '201211b_ndvi_sg',
 '201212a_ndvi_sg',
 '201212b_ndvi_sg']

In [379]:
sg_ps = ps.map(fun)

In [ ]:
img_sg = sg_ps.reduceToImage(properties=[bands_sg.get(-1)],
                             reducer=ee.Reducer.mean())\
        .reproject(images.first().projection().crs(),
                  images.first().projection().getInfo()['transform'])
Map.addLayer(img_sg, visParam, 'sg')

In [ ]:
sg_ps.first().getInfo()

In [ ]:
bands.get(0).cat('_sg').getInfo()

In [ ]:
a = ee.Array(ps.first().get('array')).toList()

In [ ]:
sg_ps.first().getInfo()

In [ ]:
from scipy.signal import savgol_filter